In [1]:
import pandas as pd
from Bio import SeqIO
from Bio import Restriction
from ntaxon.nucleotide import Sequence as Seq
from Bio import AlignIO

# Get accessions from fasta

In [2]:
fasta = "data/isolate_sequences_corrected.fasta"
seq_list = []
for record in SeqIO.parse(fasta, "fasta"):
    seq_list.append([record.id, str(record.seq)])
accessions = pd.DataFrame(data=seq_list, columns=['sample', 'sequence'])
accessions.head()

,sample,sequence
0,MN493874.1,GGTGGCGAGTGGCGGACGGGTGAGGAATACATCGGAATCTACTTTT...
1,MN493875.1,CGGAGTTAGTGGCGGACGGGTGAGTAACACGTGGGAACGTGCCTTT...
2,MN493876.1,GGTGGCGAGTGGCGGACGGGTGAGGAATACATCGGAATCTACTCTG...
3,MN493877.1,CAAGGGGAGCGGCAGACGGGTGAGTAACGCGTGGGAACGTACCTTT...
4,MN493878.1,CAAGGGGAGCGGCAGACGGGTGAGTAACGCGTGGGAACGTACCTTT...


# Get Accessions from Alignment

In [3]:
alignment = AlignIO.read("data/alignments/cropped_aln.fasta", "fasta")
print(alignment)

Alignment with 72 rows and 1335 columns
GGTGGCGAGTGGCGGACGGGTGAGGAATACATCGGAATCTACTT...ACC MN493874.1
CGGAGTTAGTGGCGGACGGGTGAGTAACACGTGGGAACGTGCCT...ACC MN493875.1
GGTGGCGAGTGGCGGACGGGTGAGGAATACATCGGAATCTACTC...ACC MN493876.1
CAAGGGGAGCGGCAGACGGGTGAGTAACGCGTGGGAACGTACCT...ACC MN493877.1
CAAGGGGAGCGGCAGACGGGTGAGTAACGCGTGGGAACGTACCT...ACC MN493878.1
CGACGTTAGCGGCGGACGGGTGAGTAACACGTGGCAACCTACCT...ACC MN493879.1
GGTGGCGAGTGGCGGACGGGGGAGGAATACATCGGAATCTACCT...ACC MN493880.1
GGCGGCGAGTGGCGGACGGGTGAGTAATATATCGGAACGTGCCC...ACC MN493881.1
GGCGGCGAGTGGCGGACGGGTGAGTAATATATCGGAACGTGCCC...ACC MN493882.1
TGATGTTAGCGGCGGATGGGTGAGTAACACGTGGCAACCTGCCC...ACC MN493883.1
GATGGTTAGCGGCGGACGGGTGAGTAACACGTAGCAACCTGCCC...ACC MN493884.1
TGATGTTAGCGGCGGACGGGTGAGTAACACGTGGTAACCTGCCT...ACC MN493885.1
GGCGGCGAGTGGCGGACGGGTGAGTAATATATCGGAACGTGCCC...ACC MN493886.1
GGCGGCGAGTGGCGGACGGGTGAGTAATATATCGGAACGTGCCC...ACC MN493887.1
TGATATTAGCGGCGGACGGGTGAGTAACACGTGGCAACCTGCCC...ACC MN493888.1
TGATATTAGCGGCGGACGGGTGAGTAACAC

In [4]:
seq_list = []
for record in alignment:
    seq_list.append([record.id, str(record.seq.ungap('-'))])
accessions = pd.DataFrame(data=seq_list, columns=['sample', 'sequence'])
accessions.head()

,sample,sequence
0,MN493874.1,GGTGGCGAGTGGCGGACGGGTGAGGAATACATCGGAATCTACTTTT...
1,MN493875.1,CGGAGTTAGTGGCGGACGGGTGAGTAACACGTGGGAACGTGCCTTT...
2,MN493876.1,GGTGGCGAGTGGCGGACGGGTGAGGAATACATCGGAATCTACTCTG...
3,MN493877.1,CAAGGGGAGCGGCAGACGGGTGAGTAACGCGTGGGAACGTACCTTT...
4,MN493878.1,CAAGGGGAGCGGCAGACGGGTGAGTAACGCGTGGGAACGTACCTTT...


In [5]:
r_map_df = pd.DataFrame(columns=['sample', 'restriction_loc'])
for i, r in accessions.iterrows():
    s = Seq(r['sequence'])
    r_maps = s.restriction_search(Restriction.MspI)
    for m in r_maps:
        r_map_df = r_map_df.append({
            'sample': r['sample'],
            'restriction_loc': m
        }, ignore_index=True)

r_map_df.head()

,sample,restriction_loc
0,MN493874.1,377
1,MN493874.1,410
2,MN493874.1,1067
3,MN493874.1,1078
4,MN493874.1,1169


In [6]:
r_map_df['value'] = 1

In [8]:
bdf = pd.crosstab(r_map_df['restriction_loc'], r_map_df['sample'])
bdf.head()

sample,MN493874.1,MN493875.1,MN493876.1,MN493877.1,MN493878.1,MN493879.1,MN493880.1,MN493881.1,MN493882.1,MN493883.1,...,MN577379.1,MN577380.1,MN577381.1,MN577382.1,MN577383.1,MN577384.1,MN577385.1,MN577386.1,MN577388.1,MN577389.1
restriction_loc,,,,,,,,,,,,,,,,,,,,,
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
63,0,0,0,0,0,1,0,0,0,1,...,1,0,0,0,0,1,0,0,0,1
64,0,0,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0
71,0,0,0,0,0,1,0,0,0,1,...,1,0,0,0,0,1,0,0,0,1
83,0,0,0,0,0,0,0,0,0,0,...,1,1,1,0,0,1,0,0,0,1


In [12]:
bdf.reset_index(drop = True)

sample,MN493874.1,MN493875.1,MN493876.1,MN493877.1,MN493878.1,MN493879.1,MN493880.1,MN493881.1,MN493882.1,MN493883.1,...,MN577379.1,MN577380.1,MN577381.1,MN577382.1,MN577383.1,MN577384.1,MN577385.1,MN577386.1,MN577388.1,MN577389.1
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
1,0,0,0,0,0,1,0,0,0,1,...,1,0,0,0,0,1,0,0,0,1
2,0,0,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0
3,0,0,0,0,0,1,0,0,0,1,...,1,0,0,0,0,1,0,0,0,1
4,0,0,0,0,0,0,0,0,0,0,...,1,1,1,0,0,1,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
97,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
98,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
99,0,0,0,0,0,0,0,0,0,0,...,0,1,1,1,0,0,0,0,0,0
100,0,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,1,0
